# Saving Toyota Data after Uniform Sampling

In [ ]:
import strym
from strym import strymread
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
import ntpath

In [ ]:
parentfolder = "../../JmscslgroupData/PandaData/"
csvlist = []
folderlist = glob.glob(parentfolder+"*")
csvlist = []
for datafolder in folderlist:
    csvlisttmp = glob.glob(datafolder+"/*.csv")
    for f in csvlisttmp:
        if "CAN" not in f:
            continue
        if "2T" in f:
            if os.path.getsize(f) >= 60*1024:
                csvlist.append(f)

In [ ]:
csvlist

In [ ]:
len(csvlist)

In [ ]:
f[1]

In [ ]:
r = strymread(csvfile=csvlist[0], dbcfile='../strym/strym/dbc/toyota_rav4_2020.dbc')
speed = r.speed()
speed

In [ ]:
speed['Time'] = speed['Time'] - speed['Time'].iloc[0]
speed

In [ ]:
from scipy.interpolate import interp1d
import plotly.io as pio

import plotly.io as pio

pio.renderers.default = 'browser'  # Forces Plotly to open in your browser

sampling_time =  1.0/20.0 # 20 Hz
t_new = np.arange(speed['Time'].iloc[0], speed['Time'].iloc[-1], sampling_time)
f = interp1d(speed['Time'].values, speed['Message'].values,  kind='zero')
v_new = f(t_new)

import plotly.graph_objects as go
# Create the plotly figure with interactive features
fig = go.Figure()

# Original data points (with markers and lines)
fig.add_trace(go.Scatter(
    x=speed['Time'], 
    y=speed['Message'],
    mode='lines+markers',
    marker=dict(size=8, color='blue'),
    line=dict(color='blue', width=2),
    name='Original Data'
))

# Interpolated data points (with markers, lines, and transparency)
fig.add_trace(go.Scatter(
    x=t_new, 
    y=v_new, 
    mode='lines+markers',
    marker=dict(size=6, color='rgba(255,0,0,0.6)'),  # Red markers with transparency
    line=dict(color='rgba(255,0,0,0.4)', width=2),   # Red lines with transparency
    name='Interpolated Data'
))

# Update layout with title, labels, and interactive features
fig.update_layout(
    title="Interpolated Speed Data with Interactive Plotting",
    xaxis_title="Time (s)",
    yaxis_title="Speed",
    hovermode="x unified"  # Enables unified hover for x-axis
)

# Show the interactive plot
fig.show()

In [ ]:
# Set the time window for plotting
time_start = 800
time_end = 1000

# Filter the data for the selected time range
mask = (speed['Time'] >= time_start) & (speed['Time'] <= time_end)
filtered_time = speed['Time'][mask]
filtered_message = speed['Message'][mask]

# Interpolation with zero-order hold
sampling_time = 1.0 / 200.0  # 200 Hz
t_new = np.arange(filtered_time.iloc[0], filtered_time.iloc[-1], sampling_time)
f = interp1d(filtered_time.values, filtered_message.values, kind='zero')
v_new = f(t_new)

# Create the plotly figure with interactive features
fig = go.Figure()

# Original data points (with markers and lines)
fig.add_trace(go.Scatter(
    x=filtered_time, 
    y=filtered_message,
    mode='lines+markers',
    marker=dict(size=8, color='blue'),
    line=dict(color='blue', width=2),
    name='Original Data'
))

# Interpolated data points (with markers, lines, and transparency)
fig.add_trace(go.Scatter(
    x=t_new, 
    y=v_new, 
    mode='lines+markers',
    marker=dict(size=6, color='rgba(255,0,0,0.6)'),  # Red markers with transparency
    line=dict(color='rgba(255,0,0,0.4)', width=2),   # Red lines with transparency
    name='Interpolated Data'
))

# Update layout with title, labels, and interactive features
fig.update_layout(
    title="Filtered Speed Data (800s to 1000s) with Interactive Plotting",
    xaxis_title="Time (s)",
    yaxis_title="Speed",
    hovermode="x unified"  # Enables unified hover for x-axis
)

# Show the interactive plot
fig.show()

In [ ]:
import ntpath
import pickle

def save_csv_data(f):

    print("------------------")
    print(f)
    print("------------------")
    r = strymread(csvfile=f, dbcfile='../strym/strym/dbc/toyota_rav4_2020.dbc')
    if not r.success:
        return

    state_var = r.state_space(rate = 20, cont_method='zero') # zero for sample and hold
    speed = r.speed()
    distance_covered  = r.integrate(speed)
    accelx = r.accelx()
    accely = r.accely()
    accelz = r.accelz()
    yaw_rate = r.yaw_rate()
    steer_rate = r.steer_rate()
    steer_angle = r.steer_angle()
    steer_fraction = r.steer_fraction()
    wheel_speed_fl = r.wheel_speed_fl()
    wheel_speed_fr = r.wheel_speed_fr()
    wheel_speed_rl = r.wheel_speed_rl()
    wheel_speed_rr = r.wheel_speed_rr()
    lead_distance = r.lead_distance()
    relative_vel = r.relative_vel()
    acc_status = r.acc_state()

    state_var.to_csv('StateData/'+ntpath.basename(r.csvfile)[0:-4]+'_20Hz'+'.csv')


    # Save the variables in a dictionary
    data = {
        'distance_covered': distance_covered,
        'accelx': accelx,
        'accely': accely,
        'accelz': accelz,
        'yaw_rate': yaw_rate,
        'steer_rate': steer_rate,
        'steer_angle': steer_angle,
        'steer_fraction': steer_fraction,
        'wheel_speed_fl': wheel_speed_fl,
        'wheel_speed_fr': wheel_speed_fr,
        'wheel_speed_rl': wheel_speed_rl,
        'wheel_speed_rr': wheel_speed_rr,
        'lead_distance': lead_distance,
        'relative_vel': relative_vel,
        'acc_status': acc_status
    }

    # Write the data to a file
    pickle_file_name = 'StateData/'+ntpath.basename(r.csvfile)[0:-4]+'_decoded'+'.pkl'
    with open(pickle_file_name, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
for f in csvlist:
    save_csv_data(f)